In [1]:
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from data_helpers import load_data, preprocess_data

path_dataset = "../data/data_train.csv"
ratings = load_data(path_dataset)

number of items: 10000, number of users: 1000


In [3]:
print(ratings.shape)

(10000, 1000)


In [ ]:
# Save ouput for submission
OUTPUT_PATH = "../data/submission.csv"
create_csv_submission(ids_test, , OUTPUT_PATH)
print("File submission.csv ready to be submitted !")